In [61]:
# Settings
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd 
import re
import time
import os

# KOBIS_TOP100.csv 만들기
* 영화 제목을 검색할 때 몇몇의 영화가 제작연도와 개봉연도가 섞여서 나타나는 문제 발생 
* 100개의 영화에 대해 수작업으로 검색시 나오는 연도를 `링크연도`로 새로운 변수를 생성

In [62]:
# 역대 박스오피스(~100) 링크연도 
data = pd.read_excel('data/KOBIS_역대_박스오피스(통합전산망_집계_기준)_2022-12-15.xlsx', header=4)
data = data[:100]
data['개봉일'] = data['개봉일'].astype('datetime64')
m_year = data['개봉일'].dt.year.astype('string').tolist()
m_year[1] = '2018'      # 네이버가 극한직업 년도 제작년도로 설정됨
m_year[9] = '2012'      # 네이버가 7번방의 선물 년도 제작년도로 설정됨
m_year[14] = '2017'     # 네이버가 신과함께-인과 연 년도 제작년도로 설정됨
m_year[25] = '2013'     # 네이버가 겨울왕국 년도 제작년도로 설정됨
m_year[27] = '2015'     # 네이버가 검사외전 년도 제작년도로 설정됨
m_year[37] = '2021'     # 네이버가 탑건: 매버릭 년도 제작년도로 설정됨
m_year[41] = '2016'     # 네이버가 공조 년도 제작년도로 설정됨
m_year[51] = '2021'     # 네이버가 한산: 용의 출현 년도 제작년도로 설정됨
m_year[54] = '2012'     # 네이버가 베를린 년도 제작년도로 설정됨
m_year[60] = '2021'     # 네이버가 공조2: 인터내셔날 년도 제작년도로 설정됨
m_year[61] = '2015'     # 네이버가 럭키 년도 제작년도로 설정됨
m_year[72] = '2014'     # 네이버가 사도 년도 제작년도로 설정됨
m_year[94] = '2017'     # 네이버가 안시성 년도 제작년도로 설정됨
m_year[98] = '2016'     # 네이버가 더 킹 년도 제작년도로 설정됨
m_list = data['영화명'].astype('string').tolist()

data['링크연도'] = m_year
data.to_csv('data/KOBIS_TOP100.csv')

# 데이터 불러오기

In [63]:
data = pd.read_csv('data/KOBIS_TOP100.csv')
m_list = data['영화명'].astype('string')
m_year = data['링크연도'].astype('string')

# 크롬 환경설정

In [64]:
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# 데이터 수집하기
## history
* 웹 화면에서 영화 제목을 불러올 때 영어 제목과 함께 출력되는 것을 발견<br>
$\rightarrow$ 혹시 모를 오류들에 대비하기 위해 영화제목은 공백과 특수문자를 제거한 후 비교
* search 화면에서 대부분의 영화는 바로 탐색되지만, 더보기를 클릭해야 탐색할 수 있는 영화를 발견<br>
$\rightarrow$ 탐색하기 전에 `더 많은 영화 보기`가 있다면 누르고 없다면 누르지 않는 코드를 추가
* 바로 리뷰를 크롤링 하려고 하면 추출되지 않는 오류가 발생<br>
$\rightarrow$ iframe을 통해 다른 사이트에서 정보를 불러오기 때문에 접근이 불가한 것. iframe의 scr로 drive를 다시 설정함.
* 한 아이디로 같은 내용이 중복으로 작성된 기록을 발견<br>
$\rightarrow$ 전의 id와 새로운 id를 비교하여 전의 리뷰의 아이디와 같으면 읽지 않는 코드를 추가



In [65]:
review_num = 10         # 각 영화당 몇 개의 리뷰를 가져올 것인가?
review_lst = []
movie_rank = 0

for movie, year in zip(m_list, m_year):
    # 네이버 영화 사이트 열기
    driver.get('https://movie.naver.com')
    
    # 검색어 입력 
    print(f'--- 영화 {movie}({year}) 을(를) 검색합니다.', end=' ')
    
    keyword = re.sub('[^0-9가-힣]', '', str(movie))       # 영화명 공백 및 특수문자 제거
    
    css_sel = '#ipt_tx_srch'
    input_box = driver.find_element(By.CSS_SELECTOR, css_sel)

    input_box.send_keys(keyword)            # 입력
    input_box.send_keys(Keys.ENTER)         # Enter
    # input_box.clear()                     # 입력된 내용 지우기
    
    time.sleep(2)
    
    # 해당 영화 사이트로 이동하기
    ## '더 많은 영화 보기'가 있으면 클릭하기
    try: 
        driver.find_element(By.CSS_SELECTOR, '#old_content > a:nth-child(5)').click() 
    except:
        pass 
    
    time.sleep(2)
    
    search_lst = driver.find_elements(By.CSS_SELECTOR, '#old_content > ul.search_list_1 > li')
    search_lst
    
    for search in search_lst:
        title_link = search.find_element(By.CSS_SELECTOR, 'dl > dt > a')
        title = re.sub('[^0-9가-힣]', "", title_link.text.split('(')[0])
        # print(title)
        title_year = search.find_elements(By.CLASS_NAME, 'etc')[0].text.split('|')[-1]
        # print(title_year, type(title_year), year, type(year))
        
        if (title == keyword) & (title_year == year):
            # print([title, year])
            title_link.click()
            print(f'>>> 해당 페이지로 이동합니다.')
            break
        
    time.sleep(2)
    # 평점 탭 클릭
    sel = '#movieEndTabMenu > li:nth-child(5) > a > em'
    driver.find_element(By.CSS_SELECTOR, sel).click()           
    
    time.sleep(1)
    # 크롤링이 가능한 접근 페이지로 다시 url 접근
    sel2 = '#pointAfterListIframe'
    review_url = driver.find_element(By.CSS_SELECTOR, sel2).get_attribute('src')
    driver.get(review_url)    

    time.sleep(1)
    # 정렬 '최신순' 선택하기
    sel3 = '#orderCheckbox > ul.sorting_list > li:nth-child(2) > a'
    driver.find_element(By.CSS_SELECTOR, sel3).click()              
    
    time.sleep(1)
    # 여러 페이지 읽기 
    review_cnt = 0

    for page in range(100):
        # 리뷰 읽기 시작 메시지 출력
        if page == 0:
            print('Start! page 1', end='')
        
        # 한 페이지의 리뷰 전체 불러오기    
        sel4 = 'body > div > div > div.score_result > ul > li'
        reviews = driver.find_elements(By.CSS_SELECTOR, sel4)

        for i, review in enumerate(reviews):
            id = review.find_element(By.CSS_SELECTOR, 'div.score_reple > dl > dt > em:nth-child(1) > a > span').text
            
            # 중복인 게시글 건너뛰기
            try:            
                if id == review_lst[-1][1]:
                    continue
            except:
                pass
            
            score = int(review.find_element(By.CSS_SELECTOR, 'div.star_score > em').text)
            text = review.find_element(By.ID, '_filtered_ment_' + str(i)).text
            
            # 텍스트가 없는 리뷰 건너 뛰기
            if text == '':
                continue
            
            # 데이터셋 만들기
            review_lst.append([movie, id, text, score])
            review_cnt += 1
            
            if review_cnt == review_num:
                break
            
        # 500개 리뷰 모이면 중단
        if review_cnt == review_num:
            print(f'({review_num}) >> End')
            break
                
        # 다음 페이지 클릭
        driver.find_element(By.ID, 'pagerTagAnchor' + str(page+2)).click()
        if (page+1) % 5 == 0:
            print(f'({review_cnt})\n> page {page+2}', end='')
        else:
            print(f'({review_cnt}) > page {page+2}', end='')
    
    
    # 영화 하나 끝남 메시지 출력
    movie_rank += 1
    print(f'박스오피스 랭킹 {movie_rank}위. {movie}의 리뷰 {review_cnt}개를 모두 수집했습니다.')
    print('-'*80)

print('모든 리뷰 수집을 완료했습니다.')


--- 영화 명량(2014) 을(를) 검색합니다. >>> 해당 페이지로 이동합니다.
Start! page 1(8) > page 2(10) >> End
박스오피스 랭킹 1위. 명량의 리뷰 10개를 모두 수집했습니다.
--------------------------------------------------------------------------------
--- 영화 극한직업(2018) 을(를) 검색합니다. >>> 해당 페이지로 이동합니다.
Start! page 1(8) > page 2(10) >> End
박스오피스 랭킹 2위. 극한직업의 리뷰 10개를 모두 수집했습니다.
--------------------------------------------------------------------------------
--- 영화 신과함께-죄와 벌(2017) 을(를) 검색합니다. >>> 해당 페이지로 이동합니다.
Start! page 1(5) > page 2(10) >> End
박스오피스 랭킹 3위. 신과함께-죄와 벌의 리뷰 10개를 모두 수집했습니다.
--------------------------------------------------------------------------------
--- 영화 국제시장(2014) 을(를) 검색합니다. >>> 해당 페이지로 이동합니다.
Start! page 1(9) > page 2(10) >> End
박스오피스 랭킹 4위. 국제시장의 리뷰 10개를 모두 수집했습니다.
--------------------------------------------------------------------------------
--- 영화 어벤져스: 엔드게임(2019) 을(를) 검색합니다. >>> 해당 페이지로 이동합니다.
Start! page 1(10) >> End
박스오피스 랭킹 5위. 어벤져스: 엔드게임의 리뷰 10개를 모두 수집했습니다.
--------------------------------------------------

In [68]:
review_data = pd.DataFrame(review_lst, columns=['movie', 'id', 'review', 'score'])
review_data.to_csv('data/review_data.csv')
review_data

,movie,id,review,score
0,명량,뮤리(cheo****),여기에 말같잖은 평 다는 놈들은 매국노 인증인듯,10
1,명량,그레이스(dbxl****),다좋았어요 한산보고나서 보니 더 좋네요,10
2,명량,영화같은삶(wh_3****),"한국영화의 전성기 시절, 전무후무한 기록을 남겨준 영화.기대했던 것보다 더 잘 만들...",10
3,명량,fml2****,2번째본 역사영화... 진짜 이순신은 대단한 영웅중에 영웅. 보는내내 끝까지 긴장놓...,10
4,명량,Blazing(seri****),정말 거짓말 안한다.클레멘타인에 대한 평가를 호평으로 오해했으면 관객수가 이리 나왔...,1
...,...,...,...,...
995,완득이,분홍이(1771****),모든분들이 연기를 잘하시네요,10
996,완득이,kitty(polo****),왜 이제 이 영화를 봤나 싶을 정도로 너무 재밌고 감동적이었다. 모든 인간은 평등하...,10
997,완득이,raca****,무엇이 차별인지 선악을 논하는 세상에서 유쾌함으로 다가가는 가장 기분좋은 방법,6
998,완득이,페르시안야옹(parr****),방금 보고 다시 또 봐도 질리지 않는 영화,10
